<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Simple_CNN_on_MNIST_dataset_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on CoLaB with pytorch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\nplease load your Google drive")
  COLAB = False
def time_fmt(t:float = 123.189)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time testing\tplease wait...\n>>>time elapse:\t{time_fmt()}")

#Assigning GPU (cuda) when available
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

Mounted at /content/drive
You are on CoLaB with pytorch version: 1.8.1+cu101
>>>time testing	please wait...
>>>time elapse:	0: 02: 03.00


In [30]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import numpy as np
import time, datetime,sys
import torch.optim as optim


In [31]:
#In this notebook we are going to train a simple convolutional neural network on mnist data using pytorch:

In [32]:
#We first defining our model class using the following codes snippets:

In [33]:
class CNNBLOCK(nn.Module):
  def __init__(self, in_channels = 1, num_classes = 10, *args, **kwargs):
    super(CNNBLOCK, self).__init__(*args, **kwargs)
    self.conv1 = nn.Conv2d(in_channels = in_channels, 
                           out_channels = 8, 
                           kernel_size = (3,3),
                           stride = (1,1),
                           padding = (1,1))
    self.maxpool = nn.MaxPool2d(kernel_size = (2,2), stride = (2,2))
    self.conv2 = nn.Conv2d(in_channels = 8, 
                           out_channels = 16,
                           kernel_size = (3,3), 
                           padding = (1,1), 
                           stride = (1,1))
    self.fc1 = nn.Linear(in_features = 16*7*7, out_features = 256)
    self.fc2 = nn.Linear(in_features = 256, out_features = num_classes)
  
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = self.maxpool(x)
    x = F.relu(self.conv2(x))
    x = self.maxpool(x)
    x = x.reshape(x.shape[0],-1)
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x


In [34]:
#Testing the class if yield what we expected:
model = CNNBLOCK()
rand_input = torch.randn(64,1,28,28)
print(f"output_shape: {model(rand_input).shape}")

output_shape: torch.Size([64, 10])


In [35]:
#Loading the MNIST data from torch library:

In [36]:
train_dataset = datasets.MNIST(root = 'train_dataset/', train = True,
                               transform = transforms.ToTensor(), download = True)
test_dataset = datasets.MNIST(root = 'test_dataset/', transform = transforms.ToTensor(),
                              train = False, download = True)

In [37]:
batch_size, epochs, learning_rate, in_channels, out_size = 64,10,0.001,1,10
train_loader = DataLoader(dataset = train_dataset,batch_size = batch_size, shuffle = True)
test_loader = DataLoader(dataset = test_dataset, batch_size = batch_size, shuffle = True)
x_train_batch, y_train_batch = next(iter(train_loader))
print(f"x_train_batch_shape: {x_train_batch.shape}, y_train_batch_shape: {y_train_batch.shape}")

x_train_batch_shape: torch.Size([64, 1, 28, 28]), y_train_batch_shape: torch.Size([64])


In [38]:
#Getting the loss function and the optimizers:
loss_obj = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

In [39]:
#The training loop:
for epoch in range(epochs):
  print(f"training starts for epoch: {epoch + 1}\n>>>please wait...\tplease wait...")
  for idx_batch, (x_train_batch, y_train_batch) in enumerate(train_loader):
    #get the data into gpu when available:
    x_train_batch.to(device = device)
    y_train_batch.to(device = device)

    #performing the forward pass here:
    preds = model(x_train_batch)
    train_loss = loss_obj(preds, y_train_batch)

    #performing the backward pass here:
    optimizer.zero_grad()
    train_loss.backward()

    #performing the gradient descent here:
    optimizer.step()

#train and performance monitoring:
def _fetch_accuracy(loader, model):
  if loader.dataset.train:
    print("checking the model accuracy for the train data")
  else:
    print('checking the model accuracy for the test data')
  
  num_correct = 0
  num_examples = 0
  model.eval()
  #we do not need to train the model again
  with torch.no_grad():
    for x, y in loader:
      x.to(device = device)
      y.to(device = device)
      preds = model(x)#this returns the predictions of shape (batch, num_classes)
      _,predictions = preds.max(1)#grab the maximum probability among the 10 classes for the batch
      num_correct+=(predictions == y).sum() #summ all true values (correct predictions)
      num_examples+=predictions.size(0)#grabs all samples in a batch (correct+incorrect predictions)
    print(f"Got: {num_correct}/{num_examples} with an accuracy of {float(num_correct)/float(num_examples)*100:.2f}")
  model.train()

_fetch_accuracy(train_loader, model)
_fetch_accuracy(test_loader, model)





training starts for epoch: 1
>>>please wait...	please wait...
training starts for epoch: 2
>>>please wait...	please wait...
training starts for epoch: 3
>>>please wait...	please wait...
training starts for epoch: 4
>>>please wait...	please wait...
training starts for epoch: 5
>>>please wait...	please wait...
training starts for epoch: 6
>>>please wait...	please wait...
training starts for epoch: 7
>>>please wait...	please wait...
training starts for epoch: 8
>>>please wait...	please wait...
training starts for epoch: 9
>>>please wait...	please wait...
training starts for epoch: 10
>>>please wait...	please wait...
checking the model accuracy for the train data
Got: 59852/60000 with an accuracy of 99.75
checking the model accuracy for the test data
Got: 9895/10000 with an accuracy of 98.95
